# Python 201.2

## Nível intermediário em Python

Os notebooks dessa segunda etapa, focam especificamente em features intermediárias/avançadas da linguagem.

Tenha em mente que algumas questões apresentadas neste notebook, farão referência aos arquivos .py encontrados dentro do diretório src no mesmo nível.

### Iterators

Iteradores em python estão por toda parte e são muito utilizados, durante este treinamento já utilizamos vários deles (como listas, tuplas e etc).

*"Em programação de computadores, um iterador se refere tanto ao objeto que permite ao programador percorrer um container, (uma coleção de elementos) particularmente listas, quanto ao padrão de projetos Iterator, no qual um iterador é usado para percorrer um container e acessar seus elementos. O padrão Iterator desacopla os algoritmos dos recipientes, porém em alguns casos, os algoritmos são necessariamente específicos dos containers e, portanto, não podem ser desacoplados."* [wiki](https://pt.wikipedia.org/wiki/Iterador)

Apesar de podermos utilizar as listas, podemos também de forma explícita percorrer as mesmas.

In [48]:
alphabet = iter('abc')
while True:
    try:
        print('it: ', next(alphabet))
    except StopIteration as e:
        print('Exception : StopIteration')
        break

it:  a
it:  b
it:  c
Exception : StopIteration


Sabendo disso, podemos permitir que uma classe sofra iteração.

In [190]:
# Implementação de Fibonacci baseada na Abordagem Iterativa
# https://pt.wikipedia.org/wiki/Sequ%C3%AAncia_de_Fibonacci#Abordagem_iterativa

class Fibonacci:
    def __init__(self, number):
        self.start(number)
    
    def start(self, number):
        self.number = number + 1
        self.__index = 0
        self.__j = 1
        self.__i = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        # Fibonacci
        t = self.__index
        # from 2 to number + 1
        if 1 < self.__index <= self.number:
            t = self.__i + self.__j
            self.__i = self.__j
            self.__j = t
        # Iterator
        if self.__index == self.number:
            raise StopIteration
        self.__index += 1
        return t

# 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144,233, 377, 610, 987
# 1597, 2584, 4181, 6765, 10946

fib = Fibonacci(21)
nums = []
for i in fib:
    nums.append(i)
print('Fibonacci Sequence: ')
print(nums)

print('-' * 20)
print('Restart and iterate over while and next:')
fib.start(8)
i = 0
while True:
    try:
        print(f'num {i}: ', next(fib))
        i += 1
    except StopIteration as si:
        print('StopIteration : exception')
        break

Fibonacci Sequence: 
[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946]
--------------------
Restart and iterate over while and next:
num 0:  0
num 1:  1
num 2:  1
num 3:  2
num 4:  3
num 5:  5
num 6:  8
num 7:  13
num 8:  21
StopIteration : exception


### [Generators](https://docs.python.org/3/tutorial/classes.html#generators)

Generators são ferramentas simples que nos permitem criar novos iteradores!

Para utilizá-los devemos utilizar a palavras reservada **yield**, acoplando isso dentro de uma função.

Os métodos **__iter__** e **__next__** são criados automaticamente quando utilizamos a criação de geradores.

E um grande diferencial, é que eles mantém o estado de execução e consomem menos memória em relação a lista, já que nem todos os elementos são previamente gerados, mas sim são gerandos em tempo de execução.

In [228]:
# gerado simples
def simple_gen(x):
    for m in range(x):
        yield m

m = simple_gen(10)
while True:
    try:
        print(next(m))
    except StopIteration as si:
        print('StopIteration : exception')
        break

0
1
2
3
4
5
6
7
8
9
StopIteration : exception


In [238]:
# Exemplo de um gerador criado de objetos
class Cat:
    def __init__(self, name):
        self.name = name

def gerador_gatos(num):
    for i in range(num):
        yield Cat(f'Cat {i}')

gc = gerador_gatos(10)
print(next(gc).name)
print(next(gc).name)

Cat 0
Cat 1


O exemplo acima é bem verboso, podemos usar algo parecido com compreensão de listas, mas para geradores.

In [244]:
gc = (Cat(f'Cat {i}') for i in range(10))
print('Type:', type(gc))
print('First Iterable item:', next(gc).name)

Type: <class 'generator'>
First Iterable item: Cat 0


### Coroutines

Diferentemente dos geradores as corotinas são generalizações de subrotinas, elas podem consumir dados.

Utilizando o operador **yield** podemos criar uma coroutine.

Normalmente, são muito utilizadas para processamento em "paralelo" (na verdade concorrência) visto que ao chamar multíplas corotinas vai se alterando a execução devido a "parada" propiciada pelo operador yield.

Nas novas versões da linguagem Python 3.5+, foi adicionado métodos assíncronos, usando async/await... na verdade as coroutines são o coração desse novo método de execução assíncrona na linguagem.

Veremos async/await mais a frente no treinamento.

In [92]:
def producer(item):
    print(f'Producing item {item}')
    yield item

def consumer():
    while True:
        item = list((yield))[0]
        print('Doing work on %s' % item)

messenger = consumer()
next(messenger)
for i in range(5):
    item = producer(f'item: {i}')
    messenger.send(item)

Producing item item: 0
Doing work on item: 0
Producing item item: 1
Doing work on item: 1
Producing item item: 2
Doing work on item: 2
Producing item item: 3
Doing work on item: 3
Producing item item: 4
Doing work on item: 4


### Programação Funcional

Apesar do Python suportar algum tipo de programação funcional, originalmente a linguagem não foi construída para este tipo de estrutura, como eu caso de Haskell, Elm, Elixir.

Entretanto, funções em Python são First-Class Citizen, existe Closure e algumas outras funções com aspecto funcional, entretanto, não existe necessariamente um paradigma funcional completo na linguagem.

Mas vamos explorar melhor os aspectos funcionais da linguagem.

#### map, reduce, filter e lambda

Dentre as funções voltadas para o paradigma funcional temos map, reduce e filter.

 - **map**: Aplica uma função a cada elemento de uma sequência.
 - **reduce**: Aplica uma função a cada elemento de uma sequência e agrega a um total.
 - **filter**: Realiza um filtro de cada elemento dentro da sequência.

Outro paradigma é a utilização de lambda expression, que é nada mais nada menos que funções anônimas de escopo restrito que aceitam argumentos e suportam apenas uma expressão.

In [11]:
# Exemplo de map
lista = [1, 2, 3, 4, 5, 6, 7]

def sqrt(x):
    return x*x

print('map : sqrt : ', list(map(sqrt, lista)))

map : sqrt :  [1, 4, 9, 16, 25, 36, 49]


In [10]:
# Exemplo de reduce
from functools import reduce

def add(x, y):
    return x + y

# (1 + 2) + (3 + 4) + (5 + 6) + 7 = 28
print('reduce = 28 : is ok? => ', reduce(add, lista) == 28)
# (1 + 2) + (3 + 4) = 10
print('reduce = 10 : is ok? => ', reduce(add, [1, 2, 3, 4]) == 10)

reduce = 28 : is ok? =>  True
reduce = 10 : is ok? =>  True


In [24]:
# Exemplo de filter
lista = [None, 1, 2, 3, None, 4]

def gt(x):
    """greater than."""
    if x:
        return x > 2
    return False

print('filter, remover None: ', list(filter(None, lista)))
print('filter, gt > 2 only:', list(filter(gt, lista)))

filter, remover None:  [1, 2, 3, 4]
filter, gt > 2 only: [3, 4]


Claro, que ao invés de definirmos expressões tão curtas no formato de funções, podemos na verdade definir expressões lambdas para isso.

In [31]:
print('sqrt: ', list(map(lambda x : x*x, [1, 2, 3, 4, 5])))

k = [[1, 2], [3, 4]]
# Colocando a função, gerada por lambda em uma variável!
sum_up = lambda x : x[0] + x[1]
r = list(map(sum_up, k))
print(f'Soma de {k}: ', r)

sqrt:  [1, 4, 9, 16, 25]
Soma de [[1, 2], [3, 4]]:  [3, 7]


#### Closure

Closure refere-se a capacidade de se criar funções dentro do escopo de outras funções encapsulando assim determinado comportamento de modo interno apenas.

Na prática Closure funciona quase como uma representação mais simples de um objeto. Em javascript nos meados da internet, essa definição de Closures foi muito utilizada (e ainda é muito ainda hoje, principalmente nos frameworks).

Closure são bem importantes, pois através desse paradigma podemos definir comportamentos a nossas funções de modo padronizado, e closure é um primeiro passo para o entendimento de decoradores.


In [39]:
def master(x):
    y = 2
    def slave():
        return x + y
    return slave

closure = master(5)
print('resultado: ', closure())

resultado:  7


### Collections

Durante muito tempo, dentro da comunidade python existiu (e muitas vezes ainda é muito mencionado nos dias atuais) a menção de que python tem baterias inclusas.

Dentre essas baterias inclusas existe o módulo de collections, o qual possui funcionalidades muitos interessantes e que podem facilitar muito a vida do desenvolvedor, para que o mesmo não precise reinvetar a roda!

Vamos apresentar alguns dos principais, mas existem muitas outras classes dentro deste módulo. [Veja todos aqui](https://docs.python.org/3.7/library/collections.html).

 - **namedtuple**
 - **Counter**

#### namedtuple

As namedtuple são estruturas bem interessantes, elas fornecem o controle das tuplas, e ao mesmo tempo permitem uma estrutura de dados melhor e mais formatada.

Muitas vezes elas são utilizadas para substituir pequenas classes que não possuem ações (métodos). Por esse mesmo motivo, em versões mais novas do Python foi implemetando os objetos com a assinatura de DataClass (que veremos mais a frente).

Mesmo assim, namedtuples podem ser uma alternativas viável e interessante!

In [254]:
from collections import namedtuple

Person = namedtuple('Person', ['name', 'age', 'gender'])
print('Type:', type(Person))

rodolfo = Person('Rodolfo', 36, 'Masculino')
print('Type:', type(rodolfo))

print(rodolfo.name)

Type: <class 'type'>
Type: <class '__main__.Person'>
Rodolfo


#### Counter

O objeto Counter permite realizar a contagem de hash.

Parece uma tarefa estranha, mas caso você possua dicionários com valores, poderia ser meio verboso e chato criar uma rotina para realizar o merge e contagem de seus valores, por este motivo o Counter pode lhe ajudar muito!!!

In [255]:
from collections import Counter

a = Counter({'a': 10, 'b': 5, 'c': 1})
b = Counter({'a': 1, 'b': 2, 'c': 0, 'd': 15})

print('Type:', type(a))
print(a + b)

Type: <class 'collections.Counter'>
Counter({'d': 15, 'a': 11, 'b': 7, 'c': 1})


### DataClass ([PEP 557](https://www.python.org/dev/peps/pep-0557))

Na nova versão da linguagem (3.7+) temos a implementação da **DataClass**, o qual cria uma **decorador** (que veremos mais a frente como utilizar), para anotar a classe e adicionar diversos métodos já implementados nela.

Para efetivamente utilizarmos esta nova feature da linguagem, precisamos também utilizar a nova feature de type annotations (PEP), a qual adicionar tipos estáticos a linguagem python. Entretanto, as type annotations servem apenas como forma de documentação e inferência de tipos das funções para linters da linguagem python acoplados dentro de IDE's (como o PyCharm).

In [26]:
from dataclasses import dataclass

# Old class to mimic DataClass
class OldPoint:
    def __init__(self, x=0.0, y=0.0):
        self.x = x
        self.y = y
    
    def __repr__(self):
        return f'{self.__class__.__name__}(x={self.x}, y={self.y})'
    
    def __eq__(self, other):
        if other.__class__ is not self.__class__:
            return NotImplemented
        return other.x == self.x and other.y == self.y

    def draw(self):
        return (self.x, self.y)

# New Python 3.7 DataClass
@dataclass
class Point:
    x: float = 0.0
    y: float = 0.0

    def draw(self):
        return f'Drawing point at ({self.x}, {self.y})'
    
        
po0, po1 = OldPoint(), OldPoint()
pn0, pn1 = Point(), Point()
print(po0)
print(pn0)
print('Equals operation: ', po0 == po1)
print('Equals operation: ', pn0 == pn1)
print('Method call:', pn0.draw())

OldPoint(x=0.0, y=0.0)
Point(x=0.0, y=0.0)
Equals operation:  True
Equals operation:  True
Method call: Drawing point at (0.0, 0.0)


Portanto, uma DataClass, já implementa para nós diversas funcionalidades, que antigamente o desenvolvedor Python deveria manualmente implementar caso quisesse possuir uma classe bem estruturada.

É possível ver mais exemplos de uso, e possíveis perigos no uso incorreto, no excelente [Blog Post](https://realpython.com/python-data-classes/) do site Real Python, assim como na [documentação oficial](https://docs.python.org/3/library/dataclasses.html) da linguagem.

In [29]:
from dataclasses import dataclass
from typing import List

@dataclass
class Carta:
    rank: str
    naipe: str

@dataclass
class Baralho:
    cartas: List[Carta]
        
baralho = Baralho([Carta('Q', 'Copa'), Carta('A', 'Espada')])
print(baralho)

Baralho(cartas=[Carta(rank='Q', naipe='Copa'), Carta(rank='A', naipe='Espada')])


### Decorators

Decorador é um design pattern em Python o qual permite que seja adicionada novas funcionalidades a um objeto existente sem modificar sua estrutura.

Como funções são cidadãos de primeira classe, elas podem ser passadas como argumentos para serem executadas por outras funções (algo parecido que vimos em Closure).

Neste sentido decoradores utilizam "closures" para trazer essa funcionalidade de uma forma simplificada.

Vejamos um simples exemplo.

In [9]:
def decorator(fn):
    def wrapper(*args, **kwargs):
        print('Before function...')
        z = fn(*args, **kwargs)
        print('After function...')
        return z
    return wrapper

def soma(x, y):
    return x + y

# Without sintatic sugar 
dec = decorator(soma)
print(dec(10, 2))

Before function...
After function...
12


Na verdade, não precisamos criar toda essa estrutura complexa para nosso decorador, em Python temos uma estrutura sintática mais simples para definir um decorador em uma função.

```python
@<nome_decorator>
```

> P.S.: Note que usamos a sintaxe \*args e \*\*kwargs, dessa maneira seria possível passar qualquer quantidade de parâmetros para nossa função, fazendo dessa maneira com que nosso decorador fosse mais genérico!

In [10]:
# Examplo
@decorator
def subtracao(x, y):
    return x - y
print(subtracao(10, 2))

Before function...
After function...
8


Para se criar uma melhor estrutura de um decorador, a melhor maneira é utilizar o pacote functools e envolver a função de wrapper com o decorador wraps. Isso permite que possamos manter a identidade de nossa função (caso ela tenha de ser inspecionada). Para isso precisamos adicionar duas novas linhas!

In [17]:
# Este import
import functools

def dec(fn):
    # Esta chamada ao decorador da functools
    @functools.wraps(fn)
    def wrapper(*args, **kwargs):
        z = fn(*args, **kwargs)
        return z
    return wrapper

@dec
def mul(x, y):
    return x * y

@decorator
def div(x, y):
    return x / y

print('Example using functools:', mul.__name__)
print('Example NOT using functools:', div.__name__)

Example using functools: mul
Example NOT using functools: wrapper


#### Múltiplos decoradores

Podemos adicionar múltiplos decoradores, neste sentido eles serão executados na ordem em que foram apresentados. Onde um decorador chama o outro e por fim a função decorada.

Por exemplo:

In [19]:
def first(fn):
    # Esta chamada ao decorador da functools
    @functools.wraps(fn)
    def wrapper(*args, **kwargs):
        print('First decorator!')
        return fn(*args, **kwargs)
    return wrapper

def second(fn):
    # Esta chamada ao decorador da functools
    @functools.wraps(fn)
    def wrapper(*args, **kwargs):
        print('Second decorator!')
        return fn(*args, **kwargs)
    return wrapper

@second
@first
def func():
    return('My function')
    
print(func())

Second decorator!
First decorator!
My function


#### Decoradores que recebem argumentos.

Outra funcionalidade bem interessante é a de adicionar parâmetros a serem recebidos nos decoradores, isso é possível, entretanto temos de mudar nossa estrutura do decorador padrão genérico e adicionar uma nova layer a ela.

In [23]:
def function(add):
    def decorator(fn):
        @functools.wraps(fn)
        def wrapper(*args, **kwargs):
            print(f'value of add param: {add}')
            return fn(*args, **kwargs) + add
        return wrapper
    return decorator

@function(add=5)
def parametros(x, y):
    return x + y

print('Total sum:', parametros(0, 0))

value of add param: 5
Total sum: 5
